In [1]:
#Importação de bibliotecas e configurções de exibição

import pandas as pd
import numpy as np

pd.set_option("display.max_columns", None)
pd.set_option("display.float_format", "{:,.2f}".format)


In [2]:
#Carregando Dataset e vizualização inicial

df = pd.read_excel("../data/online_retail_II.xlsx", sheet_name="Year 2010-2011")
df.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,"17,850.00",United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,"17,850.00",United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,"17,850.00",United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,"17,850.00",United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,"17,850.00",United Kingdom


In [3]:
#Dimensão do Dataset

df.shape

(541910, 8)

In [4]:
#Estrutura e tipo de dados

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541910 entries, 0 to 541909
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   Invoice      541910 non-null  object        
 1   StockCode    541910 non-null  object        
 2   Description  540456 non-null  object        
 3   Quantity     541910 non-null  int64         
 4   InvoiceDate  541910 non-null  datetime64[ns]
 5   Price        541910 non-null  float64       
 6   Customer ID  406830 non-null  float64       
 7   Country      541910 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 33.1+ MB


In [5]:
#Resumo estatístico

df.describe()

,Quantity,InvoiceDate,Price,Customer ID
count,"541,910.00",541910,"541,910.00","406,830.00"
mean,9.55,2011-07-04 13:35:22.342307584,4.61,"15,287.68"
min,"-80,995.00",2010-12-01 08:26:00,"-11,062.06","12,346.00"
25%,1.00,2011-03-28 11:34:00,1.25,"13,953.00"
50%,3.00,2011-07-19 17:17:00,2.08,"15,152.00"
75%,10.00,2011-10-19 11:27:00,4.13,"16,791.00"
max,"80,995.00",2011-12-09 12:50:00,"38,970.00","18,287.00"
std,218.08,NaN,96.76,"1,713.60"


In [7]:
#Lista de colunas

df.columns

Index(['Invoice', 'StockCode', 'Description', 'Quantity', 'InvoiceDate',
       'Price', 'Customer ID', 'Country'],
      dtype='object')

In [9]:
# Quantidade de valores negativos

df[df["Quantity"] < 0].shape

(10624, 8)

In [10]:
# Separação de vendas e devoluções

df_sales = df[df["Quantity"] > 0]
df_returns = df[df["Quantity"] < 0]

df_sales.shape, df_returns.shape

((531286, 8), (10624, 8))

In [12]:
# Identificação de cancelamentos (Invoice começa com "C")

df_cancel = df[df["Invoice"].astype(str).str.startswith("C")]

df_cancel.shape

(9288, 8)

In [13]:
# Quantidade de valores nulos por coluna

df.isna().sum()

Invoice             0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
Price               0
Customer ID    135080
Country             0
dtype: int64

In [16]:
# Dataset com cliente identificado

df_with_customer = df[df["Customer ID"].notna()]

# Dataset sem cliente identificado
df_without_customer = df[df["Customer ID"].isna()]

df_with_customer.shape, df_without_customer.shape

((406830, 8), (135080, 8))

In [19]:
#Criação da métrica de faturamento

df_sales["Revenue"] = df_sales["Quantity"] * df_sales["Price"]

C:\Users\wagner\AppData\Local\Temp\ipykernel_4288\240635221.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sales["Revenue"] = df_sales["Quantity"] * df_sales["Price"]


In [21]:
# Correção: Garantindo que df_sales é uma cópia independente

df_sales = df[df["Quantity"] > 0].copy()

#Criação da métrica de faturamento (forma correta)
df_sales["Revenue"] = df_sales["Quantity"] * df_sales["Price"]


In [22]:
# Visualização rápida da métrica criada

df_sales[["Quantity", "Price", "Revenue"]].head()

,Quantity,Price,Revenue
0,6,2.55,15.30
1,6,3.39,20.34
2,8,2.75,22.00
3,6,3.39,20.34
4,6,3.39,20.34


In [23]:
# Estatísticas básicas do faturamento

df_sales["Revenue"].describe()

count   531,286.00
mean         20.04
std         270.91
min     -11,062.06
25%           3.75
50%           9.90
75%          17.70
max     168,469.60
Name: Revenue, dtype: float64

In [24]:
# Refinamento: vendas válídos (quantidade e preço positivos)

df_sales = df[(df["Quantity"] > 0) & (df["Price"] > 0)].copy()

In [25]:
# Recriação da métrica de faturamento

df_sales["Revenue"] = df_sales["Quantity"] * df_sales["Price"]

In [26]:
# Nova validação estatística

df_sales["Revenue"].describe()

count   530,105.00
mean         20.12
std         270.36
min           0.00
25%           3.75
50%           9.90
75%          17.70
max     168,469.60
Name: Revenue, dtype: float64

In [27]:
# Faturamento total do período

total_revenue = df_sales["Revenue"].sum()

# Visualização formatada

f"£ {total_revenue:,.2f}"

'£ 10,666,702.54'

In [28]:
# Faturamento por produto

revenue_by_product = (
    df_sales
    .groupby("StockCode", as_index=False)["Revenue"]
    .sum()
)

In [29]:
# Ordenação decrescente

revenue_by_product = revenue_by_product.sort_values(
    by="Revenue",
    ascending=False
)

In [30]:
# Top 10 produtos por faturamento

revenue_by_product.head(10)

,StockCode,Revenue
3911,DOT,"206,248.77"
1235,22423,"174,484.74"
2390,23843,"168,469.60"
3539,85123A,"104,518.80"
2468,47566,"99,504.33"
3528,85099B,"94,340.05"
1945,23166,"81,700.92"
3914,POST,"78,119.88"
3912,M,"78,110.27"
1867,23084,"66,964.99"


In [37]:
# Faturamento por país

revenue_by_country = (
    df_sales
    .groupby("Country", as_index=False)["Revenue"]
    .sum()
)

In [38]:
# Faturamento por país

revenue_by_country = revenue_by_country.sort_values(
    by="Revenue",
    ascending=False
)

In [39]:
# Top 10 países por faturamento

revenue_by_country.head(10)

,Country,Revenue
36,United Kingdom,"9,025,222.08"
24,Netherlands,"285,446.34"
10,EIRE,"283,453.96"
14,Germany,"228,867.14"
13,France,"209,733.11"
0,Australia,"138,521.31"
31,Spain,"61,577.11"
33,Switzerland,"57,089.90"
3,Belgium,"41,196.34"
32,Sweden,"38,378.33"


In [44]:
# Faturamento e número de pedidos por país

country_metrics = (
    df_sales
    .groupby("Country")
    .agg(
        total_revenue=("Revenue", "sum"),
        total_orders=("Invoice", "nunique")
    )
    .reset_index()
)

In [45]:
# Ticket médio por país

country_metrics["avg_ticket"] = (
    country_metrics["total_revenue"] / country_metrics["total_orders"]
)

In [47]:
# Ordenação por ticket médio

country_metrics = country_metrics.sort_values(
    by="avg_ticket",
    ascending=False
)

In [48]:
country_metrics.head(10)

,Country,total_revenue,total_orders,avg_ticket
30,Singapore,"21,279.29",7,"3,039.90"
24,Netherlands,"285,446.34",94,"3,036.66"
0,Australia,"138,521.31",57,"2,430.20"
20,Japan,"37,416.37",19,"1,969.28"
21,Lebanon,"1,693.88",1,"1,693.88"
16,Hong Kong,"15,691.80",11,"1,426.53"
4,Brazil,"1,143.60",1,"1,143.60"
32,Sweden,"38,378.33",36,"1,066.06"
33,Switzerland,"57,089.90",54,"1,057.22"
9,Denmark,"18,955.34",18,"1,053.07"


In [53]:
# Vendas com cliente indentificado

df_customers = df_sales[df_sales["Customer ID"].notna()].copy()

df_customers.shape

(397885, 9)

In [54]:
# Faturamento por cliente

revenue_by_customer = (
    df_customers
    .groupby("Customer ID", as_index=False)["Revenue"]
    .sum()
)

In [56]:
# Ranking de clientes por faturamento

revenue_by_customer = revenue_by_customer.sort_values(
    by="Revenue",
    ascending=False
)

In [60]:
# Top 10 clientes por faturamento

revenue_by_customer.head(10)

,Customer ID,Revenue
1689,"14,646.00","280,206.02"
4201,"18,102.00","259,657.30"
3728,"17,450.00","194,550.79"
3008,"16,446.00","168,472.50"
1879,"14,911.00","143,825.06"
55,"12,415.00","124,914.53"
1333,"14,156.00","117,379.63"
3771,"17,511.00","91,062.38"
2702,"16,029.00","81,024.84"
0,"12,346.00","77,183.60"


In [63]:
#  Número de pedidos por cliente

orders_by_customer = (
    df_customers
    .groupby("Customer ID", as_index=False)["Invoice"]
    .nunique()
    .rename(columns={"Invoice": "total_orders"})
)

In [64]:
# Unindo faturamento e número de pedidos

customer_metrics = revenue_by_customer.merge(
    orders_by_customer,
    on="Customer ID",
    how="left"
)

In [65]:
# Ticket médio por cliente

customer_metrics["avg_ticket"] = (
    customer_metrics["Revenue"] / customer_metrics["total_orders"]
)

In [66]:
# Top 10 clientes com métricas completas

customer_metrics.head(10)

,Customer ID,Revenue,total_orders,avg_ticket
0,"14,646.00","280,206.02",73,"3,838.44"
1,"18,102.00","259,657.30",60,"4,327.62"
2,"17,450.00","194,550.79",46,"4,229.36"
3,"16,446.00","168,472.50",2,"84,236.25"
4,"14,911.00","143,825.06",201,715.55
5,"12,415.00","124,914.53",21,"5,948.31"
6,"14,156.00","117,379.63",55,"2,134.18"
7,"17,511.00","91,062.38",31,"2,937.50"
8,"16,029.00","81,024.84",63,"1,286.11"
9,"12,346.00","77,183.60",1,"77,183.60"


In [67]:
# Percentual de faturamento acumulado

customer_metrics["revenue_pct"] = (
    customer_metrics["Revenue"] / customer_metrics["Revenue"].sum()
)

customer_metrics["revenue_pct_cum"] = (
    customer_metrics["revenue_pct"].cumsum()
)

In [69]:
# Quantos clientes explicam 50%, 70%, 80% da receita

customer_metrics[
    customer_metrics["revenue_pct_cum"] <= 0.80
].shape

(1132, 6)

In [71]:
# Top 20 clientes com metricas ainda mais completas haha

customer_metrics.head(20)

,Customer ID,Revenue,total_orders,avg_ticket,revenue_pct,revenue_pct_cum
0,"14,646.00","280,206.02",73,"3,838.44",0.03,0.03
1,"18,102.00","259,657.30",60,"4,327.62",0.03,0.06
2,"17,450.00","194,550.79",46,"4,229.36",0.02,0.08
3,"16,446.00","168,472.50",2,"84,236.25",0.02,0.10
4,"14,911.00","143,825.06",201,715.55,0.02,0.12
5,"12,415.00","124,914.53",21,"5,948.31",0.01,0.13
6,"14,156.00","117,379.63",55,"2,134.18",0.01,0.14
7,"17,511.00","91,062.38",31,"2,937.50",0.01,0.15
8,"16,029.00","81,024.84",63,"1,286.11",0.01,0.16
9,"12,346.00","77,183.60",1,"77,183.60",0.01,0.17


## Conclusões da Análise

Este notebook teve como objetivo realizar uma análise exploratória e descritivade dados de vendas do varejo, buscando compreender o comportamento de faturamento, produtos, países e clientes a partir de dados transacionais reais.

### Principai achados

- O faturamento é forte e concentrado no Reino Unido, caracteristico de mercado local dominante, enquanto os mercados internacionais apresentam participação relevante, porém diluido.
- Alguns países internacionais possuem ticket médio elevado, indicando perfis de compra premium, embora com menor volume de pedidos.
- O faturamento por produto segue um padrão de concentração, com poucos itens respondendo por parcela significativa da receita, incluidno códigos não convencionais que demandam interpretação de regra de negócio.
- A análise de cliente revelou concentração de faturamento em um grupo reduzido, com perfis de clientes recorrentes e compras e compras pontuais de alto valor.
- A concentração confirmou comportamento típico de Pareto, onde uma parcela limitadad de cliente responde por uma fração relevante da receita total.

### Limitações da análise

- A análise foi restrita a dados transacionais históricos, sem aplicação de modelos preditivos ou técnicas de mineração de dados.
- Clientes sem  identificação foram excluídos da análise individual, podendo ocultar parte do comportamento agregado.
- Não foram considerados aspectos temporais avançados, como sazonalidade ou tendências ao longo do período.

### Próximos passos

Como desdobramento natural desta análise:
- Aplicar técnicas de mineração de dados, como segmentação de clientes (RFM ou Clustering), para aprofundar o entendimento dos perfis identificados.
- Avaliar estratégias distintas para mercados de alto volume e mercadaos premium.
- Explorar análises temporais para identificar padrões sazonais e tendências de crescimento.
  